# Part 3: Advanced Text Processing - LDA and BERTopic Topic Modeling (20 pts)

### **References Used:**
- LDA:
    - https://medium.com/sayahfares19/text-analysis-topic-modelling-with-spacy-gensim-4cd92ef06e06 
    - https://www.kaggle.com/code/faressayah/text-analysis-topic-modeling-with-spacy-gensim#%F0%9F%93%9A-Topic-Modeling (code for previous post)
    - https://towardsdatascience.com/topic-modelling-in-python-with-spacy-and-gensim-dc8f7748bdbf/ 
- BERTopic:
    - https://maartengr.github.io/BERTopic/getting_started/visualization/visualize_documents.html#visualize-documents-with-plotly 
    - https://maartengr.github.io/BERTopic/getting_started/visualization/visualize_topics.html
    - https://maartengr.github.io/BERTopic/getting_started/distribution/distribution.html#example
    - https://maartengr.github.io/BERTopic/getting_started/topicrepresentation/topicrepresentation.html#update-topic-representation-after-training


In [9]:
!pip install spacy
import spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 70.8 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
from tqdm import tqdm
from collections import Counter
import pandas as pd

# imports
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-dark') 

sou = pd.read_csv('data/SOTU.csv')
nlp = spacy.load("en_core_web_sm")

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
from spacy import displacy
from bertopic import BERTopic
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
import pyLDAvis.gensim_models

### LDA

To create and analyze potential topics associated with the speeches, we will first use the LDA method and package.
- Train an LDA model with 18 topics
- Output the top 10 words for each topic. 
- Output the topic distribution for the first speech
- Make a visualization

In [ ]:
def preprocess_text(text): 
    doc = nlp(text) 
    return [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and not token.is_space and len(token.lemma_) > 3]

In [ ]:
# Process all texts - note this takes ~ 5 minutes to run
processed_docs = sou['Text'].apply(preprocess_text)

In [ ]:
processed_docs

In [ ]:
# Build dictionary from processed_docs, which is a list of tokens extracted from our speeches
sou['tokens'] = processed_docs
#Gensim Dictionary object maps each word to their unique ID:
dictionary = Dictionary(sou['tokens'])
#print(dictionary.token2id)
#dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)

#create sparse vector (i, j) where i is dictionary id and j is number of occurences of that distinct word (?)
corpus = [dictionary.doc2bow(doc) for doc in sou['tokens']]

In [ ]:
# train LDA model with 18 topics
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=18, random_state=42, passes=10)

In [ ]:
# print the top 10 words for each topic
lda_model.print_topics(-1)

In [ ]:
# print the topic distribution for the first speech
sou['Text'][0]
lda_model[corpus][0]

The first speech is 99% belonging to topic 2!

In [ ]:
# make a visualization using pyLDAvis
pyLDAvis.enable_notebook()

lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_display)


In [ ]:
#save to outputs
pyLDAvis.save_html(lda_display, 'outputs/lda_topics.html')

### BERTopic
We will also conduct topic analysis using the BERTopic method and package. We will run through the following steps:
- Train a BERTopic model with a `min_topic_size` of 3 
- Output the top 10 words for each topic. 
- Output the topic distribution for the first speech
- Make a visualization of the topics

In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
docs = sou['Text'].to_list()

In [ ]:
# train the model - this takes about 30 seconds
topic_model = BERTopic(min_topic_size=3)
topics, probs = topic_model.fit_transform(docs)


# remove stop words from the topics (Hint: use CountVectorizer and then .update_topics on topic_model)
vectorizer_model = CountVectorizer(stop_words="english")
topic_model.update_topics(docs, vectorizer_model=vectorizer_model) 

In [ ]:
# output the top 10 words for each topic - hint see get_topic_info
topic_model.get_topic_info()['Representation']

In [ ]:
# output the topic distribution for the first speech
topic_distr, _ = topic_model.approximate_distribution(docs)
first_speech_viz = topic_model.visualize_distribution(topic_distr[1])

#save first speech topic distribution to outputs
first_speech_viz.write_html("outputs/BERTopic_first_speech_viz.html")
first_speech_viz

In [ ]:
# run this cell to visualize the topics
viz_topics = topic_model.visualize_topics()

#save topic visualizations to output
viz_topics.write_html("outputs/BERTopic_topics_viz.html")
viz_topics

## Discussion and Reflections

The topic distribution across the two dimensional PCA is notably different for the LDA (bag of words) and BERTopic (semantic similarity) approaches. The LDA distribution appears to have larger clusters on the right quadrant of the analyses, with significantly smaller clusters on the left quadrant. On the other hand, the BERTopic distributions land in each quadrant of the PCA grid, with more even distribution between each in terms of cluster size. This demonstrates how the two approaches use different attributes of the speeches and different algorithms to conclude topic summaries and distributions.